# 1. Text processing

We will create the pipline of text preprocessing

# 1. 1 Normalization

The first step is normalisation.
It might include:
* converting all letters to lower or upper case
* converting numbers into words or removing numbers
* removing punctuations, accent marks and other diacritics
* removing white spaces
* expanding abbreviations

In this exercise it would be ok to have a lowercase text without specific characters and digits and without unnecessery space symbols.

How neural networks could be implemented for text normalization?

In [1]:
import re
import nltk
import numpy as np
# normilize text
def normalize(text, query=False):
    text = re.sub("\d+", "", text.lower()) # Lower, no digits
    text = text.replace("_", " ") # Replace _ to space
    text = text.replace("\n", " ")
    if query:
        text = re.sub("[^\w\s*]", " ", text) # No punctuactions except *
        text = re.sub("[^a-z  *]", "", text) # Remove non english letters except *
    else:
        text = re.sub("[^\w\s]", " ", text) # No punctuactions
        text = re.sub("[^a-z ]", "", text) # Remove non english letters
    text = re.sub(' +', ' ', text) # Remove repeated whitespaces
    return text 

In [2]:
text = """Borrowed from Latin per sē (“by itself”), from per (“by, through”) and sē (“itself, himself, herself, themselves”)"""

text = normalize(text)
print(text)

borrowed from latin per s by itself from per by through and s itself himself herself themselves 


# 1.2 Tokenize
Use nltk tokenizer to tokenize the text

In [3]:
# tokenize text using nltk lib
def tokenize(text):
    return nltk.word_tokenize(text)

In [4]:
tokens = tokenize(text)
print(tokens)

['borrowed', 'from', 'latin', 'per', 's', 'by', 'itself', 'from', 'per', 'by', 'through', 'and', 's', 'itself', 'himself', 'herself', 'themselves']


# 1.3 Lemmatization
What is the difference between stemming and lemmatization?

[Optional reading](https://towardsdatascience.com/state-of-the-art-multilingual-lemmatization-f303e8ff1a8)



In [5]:
def lemmatization(tokens):
    # Lemmatize text using nltk
    lemm = nltk.stem.WordNetLemmatizer()
    return [lemm.lemmatize(token) for token in tokens]

In [6]:
lemmed = lemmatization(tokens)
print(lemmed)


['borrowed', 'from', 'latin', 'per', 's', 'by', 'itself', 'from', 'per', 'by', 'through', 'and', 's', 'itself', 'himself', 'herself', 'themselves']


# 1.4 Stop words
The next step is to remove stop words. Take the list of stop words from nltk.

In [7]:
def remove_stop_word(tokens):
    # Remove stopwords with nltk
    return [token for token in tokens if not token in nltk.corpus.stopwords.words('english')]

In [8]:
clean = remove_stop_word(lemmed)
print(clean)

['borrowed', 'latin', 'per', 'per']


# 1.5 Pipeline
Run a complete pipeline inone function.

In [9]:
text = """Borrowed from Latin per sē (“by itself”), from per (“by, through”) and sē (“itself, himself, herself, themselves”)"""

def preprocess(text):
    return remove_stop_word(lemmatization(tokenize(normalize(text)))) # All-in-one


In [10]:
clean = preprocess(text)
print(clean)

['borrowed', 'latin', 'per', 'per']


# 2. Collection

Download Reuters data from here:
https://archive.ics.uci.edu/ml/machine-learning-databases/reuters21578-mld/reuters21578.tar.gz

Read data description here:
https://archive.ics.uci.edu/ml/datasets/reuters-21578+text+categorization+collection

The function should return a list of strings - raw texts. Remove html tags using bs4 package.

## 2.1 Alternative (0.5 task bonus points)

Download songs (the process takes time, 1000 documents might be enough for a sake of exercise) from https://www.lyrics.com/. Implement a text search on it. In this case you have to creare class *Song* with fiels *title*, *artist* *and* text. The collection will contain a list of songs.

In [11]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
import json
import os
load_dir = "./loads"
lyrics_url = "https://www.lyrics.com/random.php"
error_url = "https://www.lyrics.com/no-lyrics.php"
base_url = "https://www.lyrics.com/lyric/"
n_lyrics = 1000

# Class where we store all songs
class Song:
    def __init__(self, url=None):
        self.url = url
        self.title = None
        self.text = None
        self.artists = None
        success = self.get()
        
    def __str__(self):
        return f"Song {self.title} by {', '.join(self.artists)}"
    
    def __repr__(self):
        return self.__str__()
        
    def __eq__(self, other):
        return self.url == other.url
    
    def __hash__(self):
        return self.title.__hash__()
    
    def get(self):
        # Try to get from the directory, if not - download a random song from the website
        if self.url is None or not self.load():
            if not self.download():
                raise FileNotFoundError(self.url)
            else:
                self.persist()
        print(f"Loaded {self.url}")

    def download(self):
        try:
            # Downloads a random songs from the website
            page = requests.get(lyrics_url)
            
            # Store url
            self.url = page.url
            if page.url == error_url: #If broken url - skip
                return False
            soup = BeautifulSoup(page.content, 'html.parser')
            # Get title
            self.title = soup.find(["h1","h2"], {"id":"lyric-title-text"}).text
            # Get artist name
            artists = soup.find("h3", {"class":"lyric-artist"}).findAll('a')
            self.artists = []
            for artist in artists[:-1]: # The last item is this list is "buy this song" promotions
                self.artists.append(artist.text) 
            # Get song body
            self.text = soup.find("pre", {"class":"lyric-body"}).text.replace("\r","")
        except Exception as e:
            print(e)
            return False
        return True
    
    def persist(self):
        # Try to save the song to ./load_dir/{id}
        if not os.path.exists(load_dir):
            os.mkdir(load_dir)
        file = open(os.path.join(load_dir, self.url[len(base_url):].split("/")[0]), "w")
        js = {"artists-name":self.artists,"song-name":self.title,"song-text":self.text}
        file.write(json.dumps(js))
        file.close()
        return True
    
    def load(self):
        # Try to load file from ./load_dir/{id}
        try:
            file = open(os.path.join(load_dir, self.url[len(base_url):].split("/")[0]), "r")
        except FileNotFoundError as e:
            return False
        js = json.loads(file.read())
        self.title = js["song-name"]
        self.text = js["song-text"]
        self.artists = js['artists-name']
        file.close()
        return True

In [12]:
def load_lyrics(load_existing=False):
    # loads lyrics
    songs = []
    # If we have some in directory - get them
    if load_existing and os.path.exists(load_dir):
        dir_listing = os.listdir(load_dir)
        for song in dir_listing:
            if len(songs) >= n_lyrics:
                break
            s = Song(base_url+song)
            if s not in songs:
                songs.append(s)
    # Otherwise download the remaining at random
    while len(songs) < n_lyrics:
        try:
            s = Song()
        except FileNotFoundError:
            continue
        if s not in songs:
            songs.append(s)
    return songs
    
    
def get_collection():
    collection = load_lyrics(True)
    return collection

In [13]:
collection = get_collection()
print(len(collection))

Loaded https://www.lyrics.com/lyric/1021412
Loaded https://www.lyrics.com/lyric/25292779
Loaded https://www.lyrics.com/lyric/12964046
Loaded https://www.lyrics.com/lyric/15108595
Loaded https://www.lyrics.com/lyric/7576204
Loaded https://www.lyrics.com/lyric/26570577
Loaded https://www.lyrics.com/lyric/16839229
Loaded https://www.lyrics.com/lyric/26905626
Loaded https://www.lyrics.com/lyric/23638562
Loaded https://www.lyrics.com/lyric/6933500
Loaded https://www.lyrics.com/lyric/27127930
Loaded https://www.lyrics.com/lyric/25017501
Loaded https://www.lyrics.com/lyric/14306971
Loaded https://www.lyrics.com/lyric/743122
Loaded https://www.lyrics.com/lyric/10238917
Loaded https://www.lyrics.com/lyric/12816111
Loaded https://www.lyrics.com/lyric/27184017
Loaded https://www.lyrics.com/lyric/21559417
Loaded https://www.lyrics.com/lyric/16650429
Loaded https://www.lyrics.com/lyric/15016155
Loaded https://www.lyrics.com/lyric/1904658
Loaded https://www.lyrics.com/lyric/32264362
Loaded https://w

Loaded https://www.lyrics.com/lyric/23827438
Loaded https://www.lyrics.com/lyric/18732124
Loaded https://www.lyrics.com/lyric/15811010
Loaded https://www.lyrics.com/lyric/3909604
Loaded https://www.lyrics.com/lyric/32682244
Loaded https://www.lyrics.com/lyric/5141392
Loaded https://www.lyrics.com/lyric/6647749
Loaded https://www.lyrics.com/lyric/7110418
Loaded https://www.lyrics.com/lyric/14670063
Loaded https://www.lyrics.com/lyric/30692663
Loaded https://www.lyrics.com/lyric/4594875
Loaded https://www.lyrics.com/lyric/564705
Loaded https://www.lyrics.com/lyric/30444044
Loaded https://www.lyrics.com/lyric/20840135
Loaded https://www.lyrics.com/lyric/10221694
Loaded https://www.lyrics.com/lyric/5824264
Loaded https://www.lyrics.com/lyric/4212584
Loaded https://www.lyrics.com/lyric/10063859
Loaded https://www.lyrics.com/lyric/10464531
Loaded https://www.lyrics.com/lyric/23706432
Loaded https://www.lyrics.com/lyric/4043826
Loaded https://www.lyrics.com/lyric/2800929
Loaded https://www.ly

Loaded https://www.lyrics.com/lyric/21580597
Loaded https://www.lyrics.com/lyric/17793725
Loaded https://www.lyrics.com/lyric/19784184
Loaded https://www.lyrics.com/lyric/4807440
Loaded https://www.lyrics.com/lyric/16419235
Loaded https://www.lyrics.com/lyric/26875535
Loaded https://www.lyrics.com/lyric/25093381
Loaded https://www.lyrics.com/lyric/740988
Loaded https://www.lyrics.com/lyric/32100691
Loaded https://www.lyrics.com/lyric/13870792
Loaded https://www.lyrics.com/lyric/21909084
Loaded https://www.lyrics.com/lyric/17334028
Loaded https://www.lyrics.com/lyric/26618305
Loaded https://www.lyrics.com/lyric/25962661
Loaded https://www.lyrics.com/lyric/20405671
Loaded https://www.lyrics.com/lyric/26669216
Loaded https://www.lyrics.com/lyric/22305844
Loaded https://www.lyrics.com/lyric/23788741
Loaded https://www.lyrics.com/lyric/17511140
Loaded https://www.lyrics.com/lyric/22892475
Loaded https://www.lyrics.com/lyric/22572354
Loaded https://www.lyrics.com/lyric/31250256
Loaded https:

Loaded https://www.lyrics.com/lyric/9275171
Loaded https://www.lyrics.com/lyric/94655
Loaded https://www.lyrics.com/lyric/27091284
Loaded https://www.lyrics.com/lyric/7478206
Loaded https://www.lyrics.com/lyric/19920653
Loaded https://www.lyrics.com/lyric/25569570
Loaded https://www.lyrics.com/lyric/20809842
Loaded https://www.lyrics.com/lyric/28732825
Loaded https://www.lyrics.com/lyric/18593336
Loaded https://www.lyrics.com/lyric/26361999
Loaded https://www.lyrics.com/lyric/32157598
Loaded https://www.lyrics.com/lyric/22126334
Loaded https://www.lyrics.com/lyric/28825634
Loaded https://www.lyrics.com/lyric/23650216
Loaded https://www.lyrics.com/lyric/12254610
Loaded https://www.lyrics.com/lyric/7668126
Loaded https://www.lyrics.com/lyric/9233191
Loaded https://www.lyrics.com/lyric/17324180
Loaded https://www.lyrics.com/lyric/24646314
Loaded https://www.lyrics.com/lyric/12301728
Loaded https://www.lyrics.com/lyric/1792884
Loaded https://www.lyrics.com/lyric/20219645
Loaded https://www

In [14]:
print(collection[0])

Song Angel by Anita Baker


# 3. Inverted index (old)
You will work with the boolean search model. Construct a dictionary which maps words to the postings.  

In [15]:
def make_index(collection):
    inverted_index = {}
    for song in collection:
        # Clean the text, then add words to the index
        clean = preprocess(song.text)
        for word in clean:
            if not word in inverted_index:
                inverted_index[word] = {song} # If we haven't encountered the word previously - create set with it
            else:
                inverted_index[word].add(song) # Otherwise add to the set
    return inverted_index

In [16]:
# index = make_index(collection)
# print(index['november'])

# 4. Query processing (old)

Using given search query, find all relevant documents. In binary model the relevant document is the one which contains all words from the query.

Return the list of relevant documents indexes.

In [17]:
def search(index, query):
    # Get clean query
    clean_query = preprocess(query)
    if not clean_query:
        return {} # Query empty - return nothing
    else:
        # Get first set of documents
        relevant_documents = index[clean_query[0]].copy()
    for word in clean_query[1:]:
        # Intersect with the rest of set of document in index for each word in the query
        relevant_documents &= index[word] 
    return list(relevant_documents)

In [18]:
# query = 'sound around' # change for something else if you are searching song lyrics
# relevant = search(index, query)
# print(len(relevant))
# relevant[0]

# Prefix tree

Prefix tree that we will use to create index

In [19]:
class PrefixT: 
    def __init__(self, letter, parent=None):
        self.children = {} # Children - dictionary that maps next letter to the parent
        self.l: str = letter # The content of the node, letter
        self.parent = parent # Parent of the node

    def add_word(self, word: str):
        word = word+"@" # @ will indicate end of the word
        cur_tree = self
        
        # Iteratively add every letter to the tree while going deeper
        for letter in word:
            if not letter in cur_tree.children:
                cur_tree.children[letter] = PrefixT(letter, cur_tree)
            cur_tree = cur_tree.children[letter]

    def get_words(self, cur_word=''):
        # Gets all possible words from self node, doesn't return prefix, DFS
        cur_word += self.l
        if not self.children:
            return {cur_word}
        to_return = set()
        for tree in self.children.values():
            to_return = to_return.union(tree.get_words(cur_word))
        return to_return

    def prefix(self):
        # Returns prefix (with itself) of the words 
        prefix = self.l
        if self.parent is None:
            return prefix
        else:
            return self.parent.prefix() + prefix

    def __eq__(self, other):
        return self.l == other.l

    def __hash__(self):
        return self.l.__hash__()

    def __str__(self):
        return f"PrefixT({self.l})"

    def __repr__(self):
        return self.__str__()

    def __getitem__(self, item):
        # Finds the string in the current node with name "item"
        cur_child = self
        if len(item) == 1:
            if item in self.children:
                return self.children[item]
            else:
                return None
        for i in item:
            if not i in cur_child.children:
                return None
            cur_child = cur_child[i]
        return cur_child

In [20]:
def get_rotations(word):
    # Performs all rotations 
    wordword = word+word
    to_return = set()
    length = len(word)
    
    for i in range(length):
        to_return.add(wordword[i:length+i]) # slide over the wordword and get all rotations
        
    return to_return

# Soundex index

Soundex algorithm that makes a soundex to be indexed given a word

In [21]:
soundex_dictionary = {'a':0, 'e':0, 'i':0, 'o':0, 'u':0, 'y':0, 'h':0, 'w':0,
                      'b':1, 'f':1, 'p':1, 'v':1,
                      'c':2, 'g':2, 'j':2, 'k':2, 'q':2, 's':2, 'x':2, 'z':2,
                      'd':3, 't':3, 'l':4, 'm':5, 'n':5, 'r':6}

def make_soundex(word):
    soundex = word[0].upper() # take the first letter
    
    for letter in word[1:]:
        if len(soundex) == 4: # Stop at 4 letters or when the word have been exhauseted
            break
        number = soundex_dictionary[letter]
        
        # If the number is not 0 - add it to the soundex
        if number and str(number) not in soundex:
            soundex += str(number)
    # If the length of the word isn't 4 - fill it with 0s
    if len(soundex) != 4:
        soundex +='0'*(4-len(soundex))
    return soundex

# Levenshtein Distance

Computes the distance between 2 words

In [22]:
def levenshtein_distance(word1, word2):
    matrix = np.zeros((len(word1)+1, len(word2)+1)) # Initial matrix
    
    # Fill the corner with ranges of the words
    for i in range(len(word2)+1):
        matrix[0][i] = i
    for i in range(len(word1)+1):
        matrix[i][0] = i
    
    # Go over matrix and compute the distance
    for i in range(1, len(word1)+1):
        for j in range(1, len(word2)+1):
            matrix[i][j] = min(matrix[i-1][j-1] + (1 if word1[i-1] != word2[j-1] else 0),
                               matrix[i][j-1]+1, matrix[i-1][j]+1)
    # Take lower right corner - this is an answer
    return matrix[-1][-1]

# Make Indexes

In [23]:
text = """Borrowed from Latin per sē (“by itself”), from per (“by, through”) and sē (“itself, himself, herself, themselves”)"""

def preprocess(text):
    # Now returns two lists
    # First one with lemmatization
    # Other one without lemmatization
    text = tokenize(normalize(text))
    return remove_stop_word(lemmatization(text)), remove_stop_word(text) # All-in-one


In [24]:
preprocess(text)

(['borrowed', 'latin', 'per', 'per'], ['borrowed', 'latin', 'per', 'per'])

In [25]:
def make_indexes(collection):
    inverted_index = {}
    soundex_index = {}
    prefix_tree = PrefixT('')
    for song in collection:
        
        # Clean the text, then add words to the index
        clean, unlemmatized = preprocess(song.text)
        
        for word in clean:
            if not word in inverted_index:
                inverted_index[word] = {song} # If we haven't encountered the word previously - create set with it
            else:
                inverted_index[word].add(song) # Otherwise add to the set
        
        # Now go over unlemmatized part
        for word in unlemmatized:
            
            rotations = get_rotations(word+"$")
            
            # Add rotations to the prefix tree
            for rot in rotations:
                prefix_tree.add_word(rot)
            sound = make_soundex(word)
            
            # Add sounds to the soundex index
            if not sound in soundex_index:
                soundex_index[sound] = set([word])
            else:
                soundex_index[sound].add(word)
    
    return inverted_index, soundex_index, prefix_tree

In [26]:
inverted_index, soundex_index, prefix_tree = make_indexes(collection)

# Query Preprocessing and Search

There will be two steps
1. Cleaning initial query given by user, creating a new query
2. Use cleaned query in the search

In [27]:
def rotate_back(word):
    # Rotate the word back to the original position and clean the end notifier (@)
    
    while word[-1] != "$":
        word = word[-1]+word[:-1]
        
    return word.replace("@", "")[:-1]

def wildcard_query(word, prefix_tree):
    # Perform wildcard querying (supports only single *)
    if not "*" in word:
        return None
    
    word = word+"$"
    
    # Rotate until * is in the end
    while word[-1] != "*":
        word = word[-1]+word[:-1]
    
    # Find possible words in the prefix tree (removing *)
    found = prefix_tree[word[:-1]].get_words()
    
    # Process answer
    words = list(
        map(
            lambda x: rotate_back(word[:-2]+x),
            list(found)
        )
    )
    return words

def query_prettifier(query):
    # Prettify query for the print
    query = query.copy()
    for i in range(len(query)):
        if isinstance(query[i], list):
            query[i] = f'({" || ".join(query[i])})'
    return " && ".join(query)



def parse_query(inverted_index, soundex_index, prefix_tree, query):
    # Clean the query
    clean_query = remove_stop_word(tokenize(normalize(query, True)))
    
    if not clean_query:
        return [] # Query empty - return nothing
    new_query = []
    
    for word in clean_query:
        # If the word in the index - just add it to the query
        if word in inverted_index:
            new_query.append(word)
        # If the word has *, then perform wildcard query
        # Then lemmatize the result (as we will search in the inverted_index) and remove stop words
        elif '*' in word:
            items = wildcard_query(word, prefix_tree)
            items = remove_stop_word(lemmatization(items))
            new_query.append(list(set(items)))
            
        # If this word is still unknown - try to fix it
        # find it in soundex_index the query, compute the distance for all words found and take ones with
        # minimum distance
        else:
            
            soundex = make_soundex(word)
            if soundex in soundex_index:
                sound_words = soundex_index[soundex]
                distances = []
                
                for w in sound_words:
                    distances.append(levenshtein_distance(w,word))
                
                minimum = min(distances)
                final = [sound_word for sound_word, dis in zip(sound_words, distances) if dis == minimum]
                 # Lemmatize words and remove stop words (as we will use inverted_index)
                final = remove_stop_word(lemmatization(final))
                new_query.append(list(set(final)))
            else:
                return [] # No docs with this word
    
    if not new_query:
        return [] # Empty query
    print(f"Query was perceived as: {query_prettifier(new_query)}")
    
    # Sort by strings first - lists last
    new_query = sorted(new_query, key=lambda x: isinstance(x, list))
    return new_query


def search(inverted_index, query):
    # The query has the following structure:
    # It is a list of lists or string
    # If an element of the query is a list - all docs of items of this list are OR'ed
    # And then they will be AND'ed with each other
    if not query:
        return {}
    docs = set()
    
    # Take the initial docs set
    if isinstance(new_query[0], list):
        # If the value is a list - take OR over all values
        l = new_query[0]
        docs = inverted_index[l[0]].copy()
        
        for i in l[1:]:
            docs |= inverted_index[i]
    else:
        # If the value is a string -just take it
        docs = inverted_index[new_query[0]].copy()
        
    for words in new_query[1:]:
        # Now go over remaining items
        if isinstance(words, list):
            # If the list - OR it's items and then AND with the current docs
            new_docs = inverted_index[words[0]].copy()
            for word in words[1:]:
                new_docs |= inverted_index[word]
            docs &= new_docs
            
        else:
            # If string - just AND with the current docs
            docs &=inverted_index[words]
    return docs

# Test 

In [28]:
query = "heer sound*"
new_query = parse_query(inverted_index, soundex_index, prefix_tree, query)
docs = search(inverted_index, new_query)
docs

Query was perceived as: (hear || hier) && (sound || sounding)


{Song Animals by Maroon 5,
 Song Chin Up, Cheer Up by Ryan Adams,
 Song Feel the Vibe by The Pyschotic Neurotics, Diamond D,
 Song Goofus by Slim Lamar & His Southerners,
 Song If You Know What I Mean by Neil Diamond,
 Song Lied Vom Leben by Xavas,
 Song Life by Chocolate Genius,
 Song New Kid in Town by Pickin' On,
 Song O by Omarion,
 Song Once in a Lifetime by DJ Baby Anne, Wolfsheim,
 Song Pirate Jenny [Live] by Nina Simone,
 Song Say Anything by X Japan,
 Song World of Entertainment (Woe Is Me) by Jurassic 5}

The values will not be the same with just "hear" and "sound"

In [29]:
hear_sound = inverted_index['hear'] & inverted_index['sound']
hear_sound == docs

False

As there is a song with word "hier" (German?) in the index

In [30]:
hear_sound | (inverted_index['hier'] & inverted_index['sound']) == docs

True

Another test

In [31]:
query = "heer s*nd"
new_query = parse_query(inverted_index, soundex_index, prefix_tree, query)
docs = search(inverted_index, new_query)
docs

Query was perceived as: (hear || hier) && (sound || southland || surprend || sand || sind || second || surround || stand || send || spend)


{Song Animals by Maroon 5,
 Song Burn by Dirty Halo,
 Song Chin Up, Cheer Up by Ryan Adams,
 Song Conflict by Royal Flush, Wastlanz,
 Song Cornbread by Freestyle Fellowship,
 Song Down by Low,
 Song Feel the Vibe by The Pyschotic Neurotics, Diamond D,
 Song Give It Away by Gaither Vocal Band,
 Song Goofus by Slim Lamar & His Southerners,
 Song Guitar Town by Steve Earle,
 Song If You Know What I Mean by Neil Diamond,
 Song It's up to You by The Specials,
 Song Kalamazoo by The Andrews Sisters, Glenn Miller,
 Song Lied Vom Leben by Xavas,
 Song Life by Chocolate Genius,
 Song Money Fi Spend by Vybz Kartel,
 Song New Kid in Town by Pickin' On,
 Song O Holy Night by Mormon Tabernacle Choir,
 Song O by Omarion,
 Song Once in a Lifetime by DJ Baby Anne, Wolfsheim,
 Song Perfekte Welle [Album Version] by Juli,
 Song Pirate Jenny [Live] by Nina Simone,
 Song Public Servant by Todd Rundgren,
 Song Say Anything by X Japan,
 Song Shake Some Action by Flamin' Groovies,
 Song Spend Some Time by Em